In [2]:
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_squared_error, accuracy_score
import os
import statistics
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)

Using TensorFlow backend.


In [3]:
benign_datas = pd.read_pickle('/home/sdsra/Downloads/CTU-13-Dataset/benign/datas-final-by-origin-tool.pkl')

In [4]:
benign_datas['is_malware'] = 0

In [5]:
malware_datas = pd.read_pickle('/home/sdsra/Downloads/CTU-13-Dataset/malware/1/datas-by-origin-tool.pkl')
for i in [2,3,4,5,6,7,8,9,10,11,12,13,42,43,44,45,46,47,48,49,50,52,53,54]:
    pkl_path = '/home/sdsra/Downloads/CTU-13-Dataset/malware/{}/datas-by-origin-tool.pkl'.format(str(i))
    if os.path.exists(pkl_path):
        tmp_data = pd.read_pickle(pkl_path)
        malware_datas = pd.concat([malware_datas, tmp_data], ignore_index=True)

In [6]:
path = '/home/sdsra/Downloads/CTU-13-Dataset/malware-2'
sub_dir_list = os.listdir(path)   
for dir in sub_dir_list:
    pkl_path = path +'/'+dir+'/datas-by-origin-tool.pkl'
    if os.path.exists(pkl_path):
        tmp_data = pd.read_pickle(pkl_path)
        malware_datas = pd.concat([malware_datas, tmp_data], ignore_index=True)

In [24]:
malware_datas = malware_datas[malware_datas['label'] == 'botnet']

In [25]:
malware_datas['is_malware'] = 1

In [26]:
len(benign_datas), len(malware_datas)

(6778, 72008)

In [8]:
# malware_datas = malware_datas[malware_datas['label'] == 'botnet']
upsampled_malware_datas = malware_datas.sample(len(benign_datas), replace=True)

In [9]:
upsampled_malware_datas['is_malware'] = 1

In [10]:
datas = pd.concat([benign_datas, upsampled_malware_datas], ignore_index=True)

/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [27]:
datas = pd.concat([benign_datas, malware_datas], ignore_index=True)

/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [28]:
numeric_columns = list(filter(lambda x: x not in ['src_ip','src_port','dst_ip','dst_port','proto'], benign_datas.columns))

In [30]:
datas['is_malware'].value_counts()

1    72008
0     6778
Name: is_malware, dtype: int64

In [31]:
X_train, X_test, y_train, y_test = train_test_split(datas, datas['is_malware'], test_size=0.33, random_state=42)

# cross validation

In [32]:
cls = RandomForestClassifier()

In [33]:
score = cross_validate(cls, datas[numeric_columns], datas['is_malware'], cv=10, return_train_score=False)

/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarni

In [34]:
np.mean(score['test_score'])

0.9958497239001932